In [4]:
import cv2
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle
from timeit import default_timer as timer

In [5]:
print("OpenCV version:", cv2.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Pickle module version:", pickle.format_version)

OpenCV version: 4.8.0
Pandas version: 2.1.0
NumPy version: 1.25.2


NameError: name 'matplotlib' is not defined